# 準備

## artのインストール

In [1]:
!pip3 install adversarial-robustness-toolbox

     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 34.5 MB 13 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


## ライブラリのインポート

In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()
from keras import backend as K

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [3]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR10のラベル。
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

170508288/170498071 [==============================] - 3s 0us/step


In [4]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Classifierの作成



## モデル定義

In [22]:
# モデルの定義。
def simple_model():
  inputs = Input(shape=(32, 32, 3))
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = GlobalAveragePooling2D()(x)

  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.25)(x)
  y = Dense(10, activation='softmax')(x)

  return Model(inputs, y)

model = simple_model()

# モデルのコンパイル。
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_25 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 dropout_11 (Dropout)        (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_26 (Conv2D)          (None, 16, 16, 128)       73856     
                                                           

## 学習の実行

In [23]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - ETA: 0s - loss: 2.0326 - accuracy: 0.2100

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 19s 389us/sample - loss: 2.0326 - accuracy: 0.2100 - val_loss: 1.8400 - val_accuracy: 0.3029
Epoch 2/30
50000/50000 [==============================] - 19s 381us/sample - loss: 1.6524 - accuracy: 0.3681 - val_loss: 1.5085 - val_accuracy: 0.4491
Epoch 3/30
50000/50000 [==============================] - 19s 382us/sample - loss: 1.3717 - accuracy: 0.4924 - val_loss: 1.3872 - val_accuracy: 0.5069
Epoch 4/30
50000/50000 [==============================] - 19s 382us/sample - loss: 1.1750 - accuracy: 0.5698 - val_loss: 1.1580 - val_accuracy: 0.5847
Epoch 5/30
50000/50000 [==============================] - 19s 381us/sample - loss: 1.0366 - accuracy: 0.6254 - val_loss: 1.0494 - val_accuracy: 0.6341
Epoch 6/30
50000/50000 [==============================] - 19s 381us/sample - loss: 0.9161 - accuracy: 0.6710 - val_loss: 0.9941 - val_accuracy: 0.6463
Epoch 7/30
50000/50000 [==============================] - 19s 382us/sample - loss: 0.8350 - accuracy: 0.7

## モデルの精度評価

In [24]:
model.evaluate(X_test, y_test)

[0.5955289076328277, 0.8272]

# Classifier(noisy activation)の作成

## noisy activationの作成

In [25]:
class NoisyActivation(tf.keras.layers.Layer):
  def __init__(self, activation, seed, **kwargs) :
    super().__init__(**kwargs)
    self.activation = activation
    self.seed = seed
    np.random.seed(seed=seed)
  
  def build(self, input_shape):
    self.units = input_shape[-1]
    n_init = tf.random_normal_initializer()
    self.noise = tf.Variable(
        initial_value=n_init(shape=(self.units, self.units),
                             dtype='float32'),
        trainable=True)

  def call(self, x):
    noise=np.random.rand(x.shape[1])         # ノイズ生成
    self.noise = tf.Variable(noise, dtype=tf.float32)  # テンソル演算のために変換

    return self.activation(x + self.noise)  # ノイズを加算したものを活性化関数に通して出力

## モデル定義

In [26]:
# モデルの定義。
def noisy_model():
  inputs = Input(shape=(32, 32, 3))
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = GlobalAveragePooling2D()(x)

  x = Dense(1024)(x)
  x = NoisyActivation(tf.keras.layers.ReLU(), 42)(x)
  x = Dropout(0.25)(x)
  y = Dense(10, activation='softmax')(x)

  return Model(inputs, y)

modeln = noisy_model()

# モデルのコンパイル。
modeln.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
modeln.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_30 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_31 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 dropout_14 (Dropout)        (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 16, 16, 128)       73856     
                                                           

## 学習の実行

In [27]:
# 学習の実行。
modeln.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - ETA: 0s - loss: 2.1683 - accuracy: 0.1781

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 20s 390us/sample - loss: 2.1683 - accuracy: 0.1781 - val_loss: 1.8741 - val_accuracy: 0.3031
Epoch 2/30
50000/50000 [==============================] - 19s 383us/sample - loss: 1.6789 - accuracy: 0.3642 - val_loss: 1.5312 - val_accuracy: 0.4218
Epoch 3/30
50000/50000 [==============================] - 19s 381us/sample - loss: 1.4197 - accuracy: 0.4748 - val_loss: 1.5308 - val_accuracy: 0.4174
Epoch 4/30
50000/50000 [==============================] - 19s 384us/sample - loss: 1.2405 - accuracy: 0.5445 - val_loss: 1.3481 - val_accuracy: 0.5042
Epoch 5/30
50000/50000 [==============================] - 19s 384us/sample - loss: 1.1047 - accuracy: 0.5993 - val_loss: 1.1165 - val_accuracy: 0.5983
Epoch 6/30
50000/50000 [==============================] - 19s 384us/sample - loss: 0.9846 - accuracy: 0.6465 - val_loss: 1.0899 - val_accuracy: 0.5978
Epoch 7/30
50000/50000 [==============================] - 19s 386us/sample - loss: 0.8912 - accuracy: 0.6

## モデルの精度評価

In [28]:
modeln.evaluate(X_test, y_test)

[0.6913590540885926, 0.8022]

# 敵対的攻撃

## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [29]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

### Simple

In [30]:
# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
# FGSMインスタンスの作成。
attack = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_adv = attack.generate(x=X_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


### Noisy

In [34]:
classifier_noisy = KerasClassifier(model=modeln, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_noisy = FastGradientMethod(estimator=classifier_noisy, eps=0.10, targeted=False)
X_adv_noisy = attack_noisy.generate(X_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


## 敵対的画像に対するモデルの精度評価

### Simple

In [32]:
model.evaluate(X_adv, y_test)

[8.658857342529297, 0.1051]

### Noisy

In [35]:
modeln.evaluate(X_adv_noisy, y_test)

[9.232033992004395, 0.0866]